<a href="https://colab.research.google.com/github/rebeccaastaix/BASC0033/blob/main/birth%26deathdates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm


In [ ]:
import pandas as pd
import re
import requests
import urllib.parse
from time import sleep
from tqdm import tqdm



In [ ]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]
print("Loaded file:", filename)

df = pd.read_excel(filename)
df.head()


Saving dataset_with_gender.xlsx to dataset_with_gender.xlsx
Loaded file: dataset_with_gender.xlsx


,Column,ARK,URL,Type of resource,Content type,Material type,LC number,OCLC number,Name,Dates associated with name,...,Languages,(edited) Languages,Language of original,Language of intermediate translations,Language notes,Contents,Abstract,Notes,FirstName,Gender
0,14602930,ark:/81055/vdc_00000002820C,http://access.bl.uk/item/viewer/ark:/81055/vdc...,Monograph,Language material ; Text,Online resource,NaN,NaN,"Tennyson, Alfred Tennyson, Baron",1809-1892,...,French,NaN,English,NaN,NaN,NaN,NaN,NaN,Alfred,Male
1,14603271,ark:/81055/vdc_0000000286F2,http://access.bl.uk/item/viewer/ark:/81055/vdc...,Monograph,Language material ; Text,Online resource,NaN,NaN,"Pope, Alexander",1688-1744,...,English ; French,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alexander,Male
2,14605560,ark:/81055/vdc_00000002915A,http://access.bl.uk/item/viewer/ark:/81055/vdc...,Monograph,Language material ; Text,Online resource,NaN,NaN,"Jerningham, mr (Edward)",approximately 1737-1812,...,English,English ; French,NaN,NaN,NaN,NaN,NaN,NaN,Edward,Male
3,14608735,ark:/81055/vdc_00000002C4DE,http://access.bl.uk/item/viewer/ark:/81055/vdc...,Monograph,Language material ; Text,Online resource,NaN,NaN,"Bouilly, Jean Nicolas",1763-1842,...,English,English ; French,NaN,NaN,NaN,NaN,NaN,NaN,Jean,Male
4,14608758,ark:/81055/vdc_00000002C664,http://access.bl.uk/item/viewer/ark:/81055/vdc...,Monograph,Language material ; Text,Online resource,NaN,NaN,NaN,NaN,...,French,NaN,NaN,NaN,NaN,NaN,NaN,Gothic letter ; Only 125 copies printed,NaN,Unknown


In [ ]:
# User Agent required by Wikidata
HEADERS = {
    "User-Agent": "BookMetaCollector/1.0 (contact: research@example.com)"
}

wikidata_cache = {}

def safe_get_year(claims, prop):
    """
    Safely extract a year (int) from a Wikidata property like P569 (birth) or P570 (death).
    Returns None if anything is missing / weird.
    """
    try:
        if prop not in claims:
            return None
        mainsnak = claims[prop][0]["mainsnak"]
        datavalue = mainsnak.get("datavalue")
        if not datavalue:
            return None
        time_str = datavalue["value"]["time"]
        return int(time_str[1:5])
    except Exception:
        return None


def prepare_name_for_wikidata(name):

    if not isinstance(name, str):
        return ""
    s = name.strip()
    if not s:
        return ""


    m = re.search(r'\(([^)]+)\)', s)
    if m:
        inside = m.group(1).strip()
        surname = s.split(',', 1)[0].strip()
        return f"{inside} {surname}"

    # 2) 'Last, First ...' pattern
    if ',' in s:
        parts = [p.strip() for p in s.split(',')]
        surname = parts[0]
        given_block = parts[1] if len(parts) >= 2 else ""

        junk = {
            'mr','mrs','ms','mme','madame','sir','lord','lady',
            'baron','baronne','viscountess','satirist','poet','abbé','abbe'
        }
        tokens = [t.strip(".,;") for t in given_block.split()
                  if t.strip(".,;").lower() not in junk]

        given = " ".join(tokens) if tokens else given_block

        full = given.strip()

        if surname.lower() not in full.lower():
            full = f"{full} {surname}".strip()
        return full


    return s


def wikidata_lookup(name):

    if not isinstance(name, str) or name.strip() == "":
        return (None, None)

    clean = prepare_name_for_wikidata(name)
    if not clean:
        return (None, None)


    if clean in wikidata_cache:
        return wikidata_cache[clean]


    query = urllib.parse.quote(clean)
    search_url = (
        "https://www.wikidata.org/w/api.php"
        f"?action=wbsearchentities&search={query}&language=en&format=json"
    )

    try:
        # 1) Search for the author
        sleep(0.2)
        search_resp = requests.get(search_url, headers=HEADERS)
        search_data = search_resp.json()

        if "search" not in search_data or len(search_data["search"]) == 0:
            wikidata_cache[clean] = (None, None)
            return (None, None)

        qid = search_data["search"][0]["id"]

        # 2) Full entity data
        entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
        sleep(0.2)
        entity_resp = requests.get(entity_url, headers=HEADERS)
        entity_data = entity_resp.json()

        claims = entity_data["entities"][qid].get("claims", {})

        birth = safe_get_year(claims, "P569")
        death = safe_get_year(claims, "P570")

        wikidata_cache[clean] = (birth, death)
        return (birth, death)

    except Exception:

        wikidata_cache[clean] = (None, None)
        return (None, None)


In [ ]:
sample = df['Name'].dropna().head(5).tolist()

for n in sample:
    print(
        "RAW:", n,
        "\n  CLEAN:", prepare_name_for_wikidata(n),
        "\n  WIKIDATA:", wikidata_lookup(n),
        "\n"
    )


RAW: Tennyson, Alfred Tennyson, Baron 
  CLEAN: Alfred Tennyson 
  WIKIDATA: (1809, 1892) 

RAW: Pope, Alexander 
  CLEAN: Alexander Pope 
  WIKIDATA: (1688, 1744) 

RAW: Jerningham, mr (Edward) 
  CLEAN: Edward Jerningham 
  WIKIDATA: (1774, 1822) 

RAW: Bouilly, Jean Nicolas 
  CLEAN: Jean Nicolas Bouilly 
  WIKIDATA: (1763, 1842) 

RAW: Cossé, P., satirist 
  CLEAN: P Cossé 
  WIKIDATA: (None, None) 



In [ ]:
results = []
for name in tqdm(df['Name'], desc="Fetching Wikidata info"):
    results.append(wikidata_lookup(name))

df[['BirthYear', 'DeathYear']] = pd.DataFrame(results, index=df.index)

df[['Name', 'BirthYear', 'DeathYear']].head(10)


Fetching Wikidata info: 100%|██████████| 4086/4086 [33:15<00:00,  2.05it/s]


,Name,BirthYear,DeathYear
0,"Tennyson, Alfred Tennyson, Baron",1809.0,1892.0
1,"Pope, Alexander",1688.0,1744.0
2,"Jerningham, mr (Edward)",1774.0,1822.0
3,"Bouilly, Jean Nicolas",1763.0,1842.0
4,NaN,NaN,NaN
5,"Cossé, P., satirist",NaN,NaN
6,"Driver, F. W.",NaN,NaN
7,"Gaubil, Antoine",1689.0,1759.0
8,NaN,NaN,NaN
9,"Petit, Charles, traveller",NaN,NaN


In [ ]:
results = []
for name in tqdm(df['Name'], desc="Fetching Wikidata info"):
    results.append(wikidata_lookup(name))

df[['BirthYear', 'DeathYear']] = pd.DataFrame(results, index=df.index)


Fetching Wikidata info: 100%|██████████| 4086/4086 [00:00<00:00, 293925.71it/s]


In [ ]:
filled_count = df[['BirthYear', 'DeathYear']].notna().any(axis=1).sum()
print("Number of authors with at least one date:", filled_count)


Number of authors with at least one date: 2405


In [ ]:
total = len(df)
percent = filled_count / total * 100
print(f"{filled_count} out of {total} authors ({percent:.1f}%) have at least one date.")

2405 out of 4086 authors (58.9%) have at least one date.


In [ ]:
output_name = "dataset_with_gender_birthdeath.xlsx"
df.to_excel(output_name, index=False)

from google.colab import files
files.download(output_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>